In [12]:
import h5py 
import numpy as np 
import pandas as pd 
import xgboost
from xgboost import XGBRegressor 

import pandas as pd
import numpy as np
from scipy.stats import maxwell
from numpy.random import lognormal, normal

from matplotlib import pyplot as plt 
from sklearn.metrics import mean_squared_error as mse


import pandas as pd
import shap
import sklearn

from sklearn.model_selection import train_test_split

In [62]:
#scores = model.get_booster().get_score(importance_type='gain')
from collections import defaultdict
def get_mean_importance_scores(boos):
    d = defaultdict(float)
    for b in boos:
        scores = b.get_booster().get_score(importance_type='gain')
        for k, v in scores.items():
            d[k] = d[k] + v
    for k in d:
        d[k] /= len(boos)
    return d


def get_boruta_df(X):
    rand_X = X.copy()

    for c in X.columns:
        rand_X[c] = np.random.permutation(X[c])
    
    rand_X.columns = ['rand_' + c for c in X.columns]
    return pd.concat([X, rand_X], axis=1)


def filter_features(d, cutoff=1.5, prefix='rand_'):
    filtered = set()
    for k in d:
        if k.startswith(prefix):
            continue
        if d[k] * cutoff > d[prefix + k]:
            filtered.add(k)
    return filtered


In [13]:
with h5py.File("Downloads/vigna-2021-v4-vqtl-all-utf-v2.h5", "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    keys = list(f.keys())
    d = {k: list(f[k]) for k in keys}

Keys: <KeysViewHDF5 ['doy', 'geo_id', 'gr_covar', 'gr_names', 'month', 'response_EM', 'response_R1', 'response_R3', 'response_R5', 'response_R7', 'response_R8', 'species', 'year']>


In [20]:
markers_v4 = pd.read_csv("Downloads/markers_filtered-v4.csv", index_col='Unnamed: 0')
minicore = pd.read_csv("Downloads/minicore_characterization.csv")
minicore["bVI no."] = [bytes(m.replace('\t', ''), 'utf-8') for m in minicore["VI no."]]
idx_minicore = minicore.set_index("bVI no.")


no = []

for s in d['species']:
    if s in idx_minicore.index:
        no.append(str(idx_minicore['No.'][s]))
    else:
        no.append(None)
        
        


no = np.array(no)
response = np.array([m[0] for m in d["response_R1"]])

data = pd.DataFrame(np.array([no[no != None], response[no != None]]).T
                    , columns = ['No.', 'response_R1'])


X = markers_v4.loc[data["No."]]
y = data.response_R1
X.index = np.arange(0, 1301)

In [36]:
xgb = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.01, max_delta_step=0,
       max_depth=10, min_child_weight=5, missing=1, n_estimators=200,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [39]:
boos = []
for i in range(10):
    boos.append(xgb.fit(get_boruta_df(X), y))
    print('.', end='')

..........

In [63]:
d = get_mean_importance_scores(boos)
fd = filter_features(d, cutoff=1.5, prefix='rand_')

In [69]:
boos1 = []
for i in range(10):
    boos1.append(xgb.fit(get_boruta_df(X[fd]), y))
    print('.', end='')


..........

In [73]:
d1 = get_mean_importance_scores(boos1)
fd1 = filter_features(d1, cutoff=1.5, prefix='rand_')

83

In [74]:
boos2 = []
for i in range(10):
    boos2.append(xgb.fit(get_boruta_df(X[fd1]), y))
    print('.', end='')

..........

In [75]:
d2 = get_mean_importance_scores(boos2)
fd2 = filter_features(d2, cutoff=1.5, prefix='rand_')

In [77]:
boos3 = []
for i in range(10):
    boos3.append(xgb.fit(get_boruta_df(X[fd2]), y))
    print('.', end='')

..........

In [78]:
d3 = get_mean_importance_scores(boos3)
fd3 = filter_features(d3, cutoff=1.5, prefix='rand_')

In [80]:
boos4 = []
for i in range(10):
    boos4.append(xgb.fit(get_boruta_df(X[fd3]), y))
    print('.', end='')

..........

In [81]:
d4 = get_mean_importance_scores(boos4)
fd4 = filter_features(d4, cutoff=1.5, prefix='rand_')

In [83]:
boos5 = []
for i in range(10):
    boos5.append(xgb.fit(get_boruta_df(X[fd4]), y))
    print('.', end='')

..........

In [85]:
d5 = get_mean_importance_scores(boos5)
fd5 = filter_features(d5, cutoff=1.5, prefix='rand_')

29

In [86]:
boos6 = []
for i in range(10):
    boos6.append(xgb.fit(get_boruta_df(X[fd5]), y))
    print('.', end='')

..........

In [87]:
d6 = get_mean_importance_scores(boos6)
fd6 = filter_features(d6, cutoff=1.5, prefix='rand_')

In [90]:
boos7 = []
for i in range(10):
    boos7.append(xgb.fit(get_boruta_df(X[fd6]), y))
    print('.', end='')

..........

In [92]:
d7 = get_mean_importance_scores(boos7)
fd7 = filter_features(d7, cutoff=1.5, prefix='rand_')

In [94]:
fd7

{'NC_028351.1_25184857_G_A',
 'NC_028351.1_6653772_G_A',
 'NC_028352.1_1084989_T_G',
 'NC_028352.1_21084823_T_C',
 'NC_028353.1_10441671_G_A',
 'NC_028353.1_7392850_G_C',
 'NC_028353.1_9642796_C_T',
 'NC_028354.1_16399877_G_A',
 'NC_028354.1_3627955_T_C',
 'NC_028354.1_5069290_T_A',
 'NC_028355.1_4713664_G_A',
 'NC_028356.1_2380766_G_T',
 'NC_028356.1_25365321_T_G',
 'NC_028356.1_543071_T_C',
 'NC_028357.1_51263619_T_A',
 'NC_028358.1_31321418_G_C',
 'NC_028358.1_31436226_G_A',
 'NC_028358.1_4599500_T_C',
 'NC_028359.1_10025709_C_T',
 'NC_028359.1_13497212_G_A',
 'NC_028359.1_13555726_C_G',
 'NC_028361.1_12744511_A_C',
 'NC_028361.1_8752639_A_G',
 'NC_028361.1_8933489_A_G',
 'NW_014542557.1_42048_G_C',
 'NW_014543432.1_144127_G_A'}